## Importing the necessary libraries

In [1]:
%pip install openai-clip
import torch
from torchvision import transforms
from torch.optim import Adam
from PIL import Image
import clip
import copy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=6247d556192b5527ceb0de92fda6f5d72d7cac52d2f9d20fb3e16f7cfadd3c9e
  Stored in directory: /root/.cache/pip/wheels/ab/49/bc/c2342e8e14878210ba4825cf314a53f2570f6fb18b91fce3cf
Successfully built openai-clip


## Inputs and loading models for the system

In [2]:
TEXT_PROMPT = "A potato"
LEARNING_RATE = 0.05
STEPS = 10000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Running on {DEVICE}...")
model, preprocess = clip.load("ViT-B/32", device=DEVICE)

aug_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0), ratio=(1.0, 1.0)),
    transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
])
# Normalization for CLIP
normalize = transforms.Normalize(
    mean=(0.48145466, 0.4578275, 0.40821073),
    std=(0.26862954, 0.26130258, 0.27577711)
)

Running on cuda...


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 156MiB/s]


### Image generation loop


In [ ]:
text_token = clip.tokenize([TEXT_PROMPT]).to(DEVICE)
with torch.no_grad():
    text_emb = model.encode_text(text_token)
    text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)

# Initialize Image: Start with Pure Random Noise
image = torch.randn((1, 3, 224, 224), device=DEVICE, requires_grad=True)

optimizer = Adam([image], lr=LEARNING_RATE)

# === 4. Optimization Loop ===
print(f"Optimizing noise for: '{TEXT_PROMPT}'")

for step in range(STEPS):
    optimizer.zero_grad()
    total_loss = 0
    for _ in range(4):
        # aug_img = aug_transform(image)
        aug_img = image
        # Normalize and Encode
        image_norm = normalize(aug_img)
        img_emb = model.encode_image(image_norm)
        img_emb = img_emb / img_emb.norm(dim=-1, keepdim=True)

        # Loss: 1 - Similarity
        # We multiply by a large factor (10) to force the optimizer to push harder
        loss = (1 - torch.cosine_similarity(img_emb, text_emb).mean())
        total_loss += loss

    # Average the loss over the augmentations
    final_loss = total_loss / 4
    final_loss.backward()

    optimizer.step()

    # Clamp to valid image range (0-1)
    with torch.no_grad():
        image.data.clamp_(0, 1)

    if step % 100 == 0:
        # Check the RAW similarity (without augmentation) to see true progress
        with torch.no_grad():
            raw_emb = model.encode_image(normalize(image))
            raw_emb /= raw_emb.norm(dim=-1, keepdim=True)
            raw_sim = torch.cosine_similarity(raw_emb, text_emb).item()

        print(f"Step {step} | Loss: {final_loss.item():.4f} | Current Raw Score: {raw_sim:.4f}")



## Saving the image

In [ ]:
print("\n--- Saving ---")
# Save the final robust noise
out_img = image.detach().cpu().squeeze()
out_pil = transforms.ToPILImage()(out_img)
out_pil.save("robust_fooling_noise.png")

## Verifying the similarity using CLIP

In [ ]:
verify_img = preprocess(Image.open("robust_fooling_noise.png")).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    v_img_feat = model.encode_image(verify_img)
    v_img_feat /= v_img_feat.norm(dim=-1, keepdim=True)

    final_sim = torch.cosine_similarity(v_img_feat, text_emb).item()

print(f"Final Similarity Score after Saving: {final_sim:.4f}")